<a href="https://colab.research.google.com/github/ZongoSosthene/PROJET_DDEFI_2025/blob/main/backtest_4_facteurs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance yesg PyPortfolioOpt portfolio-backtest riskfolio-lib plotly scikit-learn

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.1/985.1 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 16.3 MB/s eta 0:00:00
  Created wheel for yesg: filename=yesg-2.1.1-py3-none-any.whl size=6105 sha256=6bffc16b16b92cbf1e33eed232544d2bfde4a121cb1f4946cc379819106e1a57
  Stored in directory: /root/.cache/pip/wheels/2b/53/0e/129a8d67d809bf89a196f3ea3e56b9df3da4975924157cb1a3
Successfully built yesg


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import scipy.optimize as opt
import yesg
#import plotly.express as px
#import seaborn as sns
import plotly.graph_objects as go
#from plotly.subplots import make_subplots
#import riskfolio as rp
import warnings
import requests
from io import BytesIO, StringIO
import pypfopt as pf
#warnings.filterwarnings("ignore")

In [ ]:
# Function to calculate the cost of transactions
def transaction_costs(w, w0, TC):
    return np.sum(TC * np.abs(w - w0)**2)

# Modified objective function to include transaction costs
def objective(w, cov_mat, w0, TC, factor_TC):
    # Original variance-based function
    s = 0
    for i in range(len(w)):
        for j in range(len(w)):
            s += (w[i] * (cov_mat @ w)[i] - w[j] * (cov_mat @ w)[j])**2
    # Add transaction costs as a penalty term
    return s + factor_TC * transaction_costs(w, w0, TC)

def rp_weights(df, w0= np.array([0.2, 0.2, 0.2, 0.2, 0.2]), TC = np.array([0.0005, 0.0005, 0.0005, 0.0005, 0.0005]), factor_TC = .0001):
    """
    Function to calculate the weights for the risk parity portfolio
    df: DataFrame with the ETF returns
    w0: Initial weights (current portfolio)
    TC: Transaction costs (5 bps)
    factor_TC: Scaling factor for transaction costs (tune this as needed)
    """
    cov_mat = df.cov().values  # Covariance matrix of ETF returns

    # Constraints: weights sum to 1 and are long-only
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # Sum of weights = 1
                   {'type': 'ineq', 'fun': lambda w: w}]            # Weights must be positive (long only)

    # Optimize
    result = opt.minimize(objective, w0, args=(cov_mat, w0, TC, factor_TC), constraints=constraints, tol=1e-20)
    w_opt = result.x  # Optimized weights
    return w_opt


def rp_weights_1(df, w0= np.array([0.25, 0.25, 0.25, 0.25]), TC = np.array([0.0005, 0.0005, 0.0005, 0.0005]), factor_TC = .0001):
    """
    Function to calculate the weights for the risk parity portfolio
    df: DataFrame with the ETF returns
    w0: Initial weights (current portfolio)
    TC: Transaction costs (5 bps)
    factor_TC: Scaling factor for transaction costs (tune this as needed)
    """
    cov_mat = df.cov().values  # Covariance matrix of ETF returns

    # Constraints: weights sum to 1 and are long-only
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # Sum of weights = 1
                   {'type': 'ineq', 'fun': lambda w: w}]            # Weights must be positive (long only)

    # Optimize
    result = opt.minimize(objective, w0, args=(cov_mat, w0, TC, factor_TC), constraints=constraints, tol=1e-20)
    w_opt = result.x  # Optimized weights
    return w_opt

Minimum variance portfolio weight optimization


In [ ]:
def min_var_weights(train_data):
    """Effectue l'optimisation Min-Var et retourne les poids optimaux."""
    try:
        mean_returns = train_data.mean()
        cov_matrix = train_data.cov()
        ef = pf.EfficientFrontier(mean_returns, cov_matrix)
        weights = ef.min_volatility()
        return np.array(list(weights.values()))
    except Exception as e:
        print(f"Erreur lors du calcul des poids du min-var portfolio : {e}")
        return None

In [ ]:
def calculate_metrics(portfolio_returns, spy_returns, slice):
    """Calcule les métriques de performance pour une tranche donnée."""
    try:
        cumulative_return = float((1 + portfolio_returns).prod() - 1)
        cov_matrix = np.cov(portfolio_returns, spy_returns)
        beta = cov_matrix[0, 1] / cov_matrix[1, 1]
        expected_annual_return = float(portfolio_returns.mean() * 252)
        volatility = float(portfolio_returns.std() * np.sqrt(i))
        sharpe_ratio = float(cumulative_return / volatility)
        treynor_ratio = float(cumulative_return / beta)


        return {
            "Slice": slice,
            "Expected Annual Return": expected_annual_return,
            "Volatility": volatility,
            "Sharpe Ratio": sharpe_ratio,
            "Treynor Ratio": treynor_ratio,
            "Cumulative Return": cumulative_return
        }
    except Exception as e:
        print(f"Erreur lors du calcul des métriques de la tranche {i} : {e}")
        return None


In [ ]:
def get_data(tickers, start_date, end_date):
    try:
        data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
        returns = data.pct_change().dropna()

        spy_data = yf.download('SPY', start=start_date, end=end_date)['Adj Close']
        spy_returns = spy_data.pct_change().dropna()
    except Exception as e:
        print(f"Erreur lors de la récupération des données : {e}")
        return
    return returns, spy_returns


In [ ]:
def convert_metrics(metrics_list):
    try:
        metrics_df = pd.DataFrame(metrics_list)
        metrics_df = metrics_df.set_index("Slice")
    except KeyError as e:
        print(f"Erreur lors de la conversion en DataFrame : {e}")
        return
    except Exception as e:
        print(f"Erreur inattendue : {e}")
        return
    return metrics_df


In [ ]:
def backtest(i, j, k, start_date, end_date, tickers, strategy):

  # Désactiver les warnings pour les matrices non définies positives
    warnings.filterwarnings("ignore", message="You must convert self.cov to a positive definite matrix")
    try:
        # Vérification des paramètres d'entrée
        if j < i:
            raise ValueError("Le deuxième paramètre (j) doit être supérieur au premier (i).")
    except ValueError as e:
        print(f"Erreur d'entrée : {e}")
        return

    # Étape 1 : Récupération des données
    returns, spy_returns = get_data(tickers, start_date, end_date)

    # Étape 2 : Initialisation des variables
    try:
        train_data = returns.iloc[:j, :]
        test_data = returns.iloc[j:, :]
        spy_test_data = spy_returns.iloc[j:]
        metrics_list = []
        start_idx = 0
        weights_list = []
    except Exception as e:
        print(f"Erreur lors de l'initialisation des variables : {e}")
        return

    # Étape 3 : Optimisation RP pour chaque tranche
    try:
        while start_idx + i <= len(test_data) and start_idx + i <= len(spy_test_data) :
            # Etape 3 : Découpage des données de test
            slice_data = test_data.iloc[start_idx:start_idx + i]
            spy_slice_data = spy_test_data.iloc[start_idx:start_idx + i].to_numpy().flatten()

            # Etape 4 : Calcul des poids du portefeuille
            if strategy=="ERC":
                # Optimisation RP sur les données d'entraînement actuelles
                #port = rp.Portfolio(returns=train_data)
                #port.assets_stats(method_mu='hist', method_cov='hist')
                #weights = port.rp_optimization(model='Classic', rm='MV').to_numpy().flatten()
                try :
                    weights = rp_weights(train_data)
                except:
                    weights = rp_weights_1(train_data)

                weights_list.append(weights)
                portfolio_returns = slice_data @ weights    # Calcul des rendements du portefeuille pour la tranche

            elif strategy=="Equal weight":
                weights = np.array([1/len(tickers)]*len(tickers))
                weights_list.append(weights)
                portfolio_returns = slice_data @ weights

            elif strategy=="Benchmark":
                portfolio_returns = spy_slice_data

            elif strategy=="Min-var":
                weights = min_var_weights(train_data)
                weights_list.append(weights)
                portfolio_returns = slice_data @ weights

            else:
                print("Stratégie non reconnue, veuillez choisir entre 'ERC', 'Equal weight', 'Benchmark' et 'Min-var'")
                return

            # Étape 5 : Calcul des métriques
            metrics_list.append(calculate_metrics(portfolio_returns, spy_slice_data, len(metrics_list) + 1))

            # Mise à jour des données d'entraînement
            train_data = pd.concat([train_data, test_data.iloc[start_idx:start_idx + k]]).iloc[-j:]
            start_idx += k

    except Exception as e:
        print(f"Erreur lors de l'optimisation ou du calcul des métriques : {e}")
        return

    # Étape 6 : Conversion en DataFrame
    metrics_df = convert_metrics(metrics_list)

    return metrics_df, weights_list


In [ ]:
tickers = ['SPHQ', 'IVE', 'SPYD', 'SPLV', 'SPMO']
start_date = '2018-01-01'
end_date = '2024-11-01'
i = 5
j = 22
k = i
strategies = ["Equal weight", "Benchmark", "Min-var", "ERC"]
sharpe_ratios = pd.DataFrame(columns=strategies)
cumulative_returns = pd.DataFrame(columns=strategies)
for strategy in strategies:
    metrics_df, weights_list = backtest(i, j, k, start_date, end_date, tickers, strategy)
    sharpe_ratios[strategy] = metrics_df["Sharpe Ratio"]
    cumulative_returns[strategy] = metrics_df["Cumulative Return"]

[*********************100%***********************]  5 of 5 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  5 of 5 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  5 of 5 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  5 of 5 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
def plot_data(data):
    # Create a figure
    fig = go.Figure()

    # Loop through each column in the DataFrame and add a trace for each
    for column in data.columns:
        fig.add_trace(go.Scatter(x=data.index, y=data[column], mode='lines', name=column))

    # Show the plot
    fig.show()

def box_plot_data(data):
    fig = go.Figure()

    for column in data.columns:
        fig.add_trace(go.Box(y=data[column], name=column))

    fig.update_layout(
        title="Box Plot",
        yaxis_title="Values",
        xaxis_title="Columns",
    )

    fig.show()

def plot_moving_average_data(data):
    # Apply a rolling mean (moving average) for smoothing
    smoothed_sr = data.rolling(window=30, min_periods=1).mean()  # 5-period rolling average

    fig = go.Figure()

    for column in smoothed_sr.columns:
        fig.add_trace(go.Scatter(x=smoothed_sr.index, y=smoothed_sr[column], mode='lines', name=column))

    # Update layout for better visualization
    fig.update_layout(
        title="Smoothed Trends of Each Column",
        xaxis_title="Index",
        yaxis_title="Smoothed Values",
    )

    fig.show()

In [ ]:
sharpe_ratios.describe()
plot_data(sharpe_ratios)
box_plot_data(sharpe_ratios)
plot_moving_average_data(sharpe_ratios)

## Sans qualité

In [ ]:
tickers = ['IVE', 'SPYD', 'SPLV', 'SPMO']
start_date = '2018-01-01'
end_date = '2024-11-01'
i = 3
j = 18  # 22 initialement
k = i
strategies = ["Equal weight", "Benchmark", "Min-var", "ERC"]
sharpe_ratios = pd.DataFrame(columns=strategies)
cumulative_returns = pd.DataFrame(columns=strategies)
for strategy in strategies:
    metrics_df, weights_list = backtest(i, j, k, start_date, end_date, tickers, strategy)
    sharpe_ratios[strategy] = metrics_df["Sharpe Ratio"]
    cumulative_returns[strategy] = metrics_df["Cumulative Return"]


sharpe_ratios.describe()
plot_data(sharpe_ratios)
box_plot_data(sharpe_ratios)
plot_moving_average_data(sharpe_ratios)

[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed


##Sans dividend


In [ ]:
tickers = ['SPHQ', 'IVE', 'SPLV', 'SPMO']
start_date = '2018-01-01'
end_date = '2024-11-01'
i = 6
j = 16
k = i
strategies = ["Equal weight", "Benchmark", "Min-var", "ERC"]
sharpe_ratios = pd.DataFrame(columns=strategies)
cumulative_returns = pd.DataFrame(columns=strategies)
for strategy in strategies:
    metrics_df, weights_list = backtest(i, j, k, start_date, end_date, tickers, strategy)
    sharpe_ratios[strategy] = metrics_df["Sharpe Ratio"]
    cumulative_returns[strategy] = metrics_df["Cumulative Return"]


sharpe_ratios.describe()
plot_data(sharpe_ratios)
box_plot_data(sharpe_ratios)
plot_moving_average_data(sharpe_ratios)

[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed


## Sans Volatilité

In [ ]:
tickers = ['SPHQ', 'IVE', 'SPYD','SPMO']
start_date = '2018-01-01'
end_date = '2024-11-01'
i = 6
j = 16
k = i
strategies = ["Equal weight", "Benchmark", "Min-var", "ERC"]
sharpe_ratios = pd.DataFrame(columns=strategies)
cumulative_returns = pd.DataFrame(columns=strategies)
for strategy in strategies:
    metrics_df, weights_list = backtest(i, j, k, start_date, end_date, tickers, strategy)
    sharpe_ratios[strategy] = metrics_df["Sharpe Ratio"]
    cumulative_returns[strategy] = metrics_df["Cumulative Return"]


sharpe_ratios.describe()
plot_data(sharpe_ratios)
box_plot_data(sharpe_ratios)
plot_moving_average_data(sharpe_ratios)

[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed


### Sans Value

In [ ]:
tickers = ['SPHQ', 'SPYD', 'SPLV', 'SPMO']
start_date = '2018-01-01'
end_date = '2024-11-01'
i = 7
j = 16
k = i
strategies = ["Equal weight", "Benchmark", "Min-var", "ERC"]
sharpe_ratios = pd.DataFrame(columns=strategies)
cumulative_returns = pd.DataFrame(columns=strategies)
for strategy in strategies:
    metrics_df, weights_list = backtest(i, j, k, start_date, end_date, tickers, strategy)
    sharpe_ratios[strategy] = metrics_df["Sharpe Ratio"]
    cumulative_returns[strategy] = metrics_df["Cumulative Return"]



sharpe_ratios.describe()
plot_data(sharpe_ratios)
box_plot_data(sharpe_ratios)
plot_moving_average_data(sharpe_ratios)

[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed


## Sans Momentum

In [ ]:
tickers = ['SPHQ', 'IVE', 'SPYD', 'SPLV']
start_date = '2018-01-01'
end_date = '2024-11-01'
i = 5
j = 20
k = i
strategies = ["Equal weight", "Benchmark", "Min-var", "ERC"]
sharpe_ratios = pd.DataFrame(columns=strategies)
cumulative_returns = pd.DataFrame(columns=strategies)
for strategy in strategies:
    metrics_df, weights_list = backtest(i, j, k, start_date, end_date, tickers, strategy)
    sharpe_ratios[strategy] = metrics_df["Sharpe Ratio"]
    cumulative_returns[strategy] = metrics_df["Cumulative Return"]

sharpe_ratios.describe()
plot_data(sharpe_ratios)
box_plot_data(sharpe_ratios)
plot_moving_average_data(sharpe_ratios)

[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed
